<img src="tmva_logo.gif" height="20%" width="20%">

# TMVA Classification Example

## Declare Factory

Create the Factory class. Later you can choose the methods
whose performance you'd like to investigate. 

The factory is the major TMVA object you have to interact with. Here is the list of parameters you need to pass

 - The first argument is the base of the name of all the output
weightfiles in the directory weight/ that will be created with the 
method parameters 

 - The second argument is the output file for the training results
  
 - The third argument is a string option defining some general configuration for the TMVA session. For example all TMVA output can be suppressed by removing the "!" (not) in front of the "Silent" argument in the option string

In [1]:
TMVA::Tools::Instance();


auto outputFile = TFile::Open("Higgs_ClassificationOutputVBFbkg.root", "RECREATE");

TMVA::Factory factory("TMVA_Higgs_Classification", outputFile,
                      "!V:ROC:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" ); 

## Setup Dataset(s)

Define input data file and signal and background trees

In [2]:
TString inputFileName1 = "vbf_train.root";
TString inputFileName2 = "vbf_test.root";
TString inputFileName3 = "bkg_train.root";
TString inputFileName4 = "bkg_test.root";

//TString inputFileName = "tmva_class_example.root";

auto inputFile1 = TFile::Open( inputFileName1 );
auto inputFile2 = TFile::Open( inputFileName2 );
auto inputFile3 = TFile::Open( inputFileName3 );
auto inputFile4 = TFile::Open( inputFileName4 );


// --- Register the training and test trees

//TTree *signalTree     = (TTree*)inputFile1->Get("TF");
//TTree *backgroundTree = (TTree*)inputFile2->Get("TF");

TTree *signalTreeTrain     = (TTree*)inputFile1->Get("test");
TTree *signalTreeTest     = (TTree*)inputFile2->Get("test");
TTree *backgroundTreeTrain = (TTree*)inputFile3->Get("test");
TTree *backgroundTreeTest = (TTree*)inputFile4->Get("test");


signalTreeTrain->Print();
signalTreeTest->Print();
backgroundTreeTrain->Print();
backgroundTreeTest->Print();

 TH1F *hist1=new TH1F("hist_bkg","",1,-100000000000000000,10000000000000);
 backgroundTreeTest->Draw("weight>>hist_bkg");
 double weight_bkg_total = hist1->GetMean()*hist1->GetEntries();

  string weight_bkg = "weight/"+ std::to_string(weight_bkg_total);

  cout<<" w bkg:   "<<weight_bkg <<endl;

 TH1F *hist2=new TH1F("hist_sig","",1,-100000000000000000,10000000000000);
 signalTreeTest->Draw("weight>>hist_sig");
 double weight_sig_total = hist2->GetMean()*hist2->GetEntries();

  string weight_sig = "weight/"+ std::to_string(weight_sig_total);

  cout<<" w sig:   "<<weight_sig <<endl;



******************************************************************************
*Tree    :test      : bbll_ee                                                *
*Entries :     8990 : Total =         3197558 bytes  File  Size =    2415946 *
*        :          : Tree compression factor =   1.32                       *
******************************************************************************
*Br    0 :BDTvb     : BDTvb/D                                                *
*Entries :     8990 : Total  Size=      72623 bytes  File Size  =      69290 *
*Baskets :        3 : Basket Size=      32000 bytes  Compression=   1.04     *
*............................................................................*
*Br    1 :BDTvg     : BDTvg/D                                                *
*Entries :     8990 : Total  Size=      72623 bytes  File Size  =      69276 *
*Baskets :        3 : Basket Size=      32000 bytes  Compression=   1.04     *
*...................................................

Info in <TCanvas::MakeDefCanvas>:  created default TCanvas with name c1


## Declare DataLoader(s)

The next step is to declare the DataLoader class that deals with input variables 

Define the input variables that shall be used for the MVA training
note that you may also use variable expressions, which can be parsed by TTree::Draw( "expression" )]

In [3]:
TMVA::DataLoader * loader = new TMVA::DataLoader("dataset");

loader->AddVariable("DiLeptonMass");
loader->AddVariable("DiLeptonPt");
//loader->AddVariable("DiLeptonDeltaR");
loader->AddVariable("DiLeptonDeltaPhi");
loader->AddVariable("MjjMax");
loader->AddVariable("deltaetajj");
loader->AddVariable("pTjjMax");
loader->AddVariable("deltaphijjmax");

loader->AddVariable("Lepton1Pt");
loader->AddVariable("Lepton1Eta");
//loader->AddVariable("Lepton1Phi");
loader->AddVariable("BJet1Pt");
loader->AddVariable("BJet1Eta");
//loader->AddVariable("BJet1Phi");

//loader->AddVariable("Lepton0Pt");
loader->AddVariable("Lepton0Eta");
//loader->AddVariable("Lepton0Phi");
//loader->AddVariable("BJet0Pt");
loader->AddVariable("BJet0Eta", "BJet0Eta", "", 'F');
//loader->AddVariable("BJet0Phi");
loader->AddVariable("Mtot");
loader->AddVariable("MTHiggs");
loader->AddVariable("DiBJetMass");
loader->AddVariable("DiBJetPt");
loader->AddVariable("DiBJetDeltaR");
loader->AddVariable("DiBJetDeltaPhi");
loader->AddVariable("minDRlj1");
loader->AddVariable("minDRlj2");
//loader->AddVariable("sumC");
//loader->AddVariable("Mjl");
loader->AddVariable("pTtot");
loader->AddVariable("minDbl");
loader->AddVariable("KLF_mtop");


loader->AddVariable("NOtherJets");
//loader->AddVariable("NForwardJets");


loader->AddSpectator("eventNumber", "eventNumber", "" );


cout<<"ok  "<<endl;


ok  


We set now the input data trees in the TMVA DataLoader class

In [4]:

// global event weights (see below for setting event-wise weights)
Double_t weight2 = 1.0;
//TCut cutstr = "fmod(eventNumber, 2) == 0.";
//TCut cutste = "fmod(eventNumber, 2) == 1.";

//TCut cutstr = "BJet0Eta>0";
//TCut cutste = "BJet0Eta<0";

TCut cutstr = "";
TCut cutste = "";

//TCut cutstr = "(eventNumber%2)==0";
//TCut cutste = "(eventNumber%2)==1";

loader->AddTree (signalTreeTrain,        TString("Signal"), weight2, cutstr, TMVA::Types::kTraining);
loader->AddTree (signalTreeTest,          TString("Signal"), weight2, cutste, TMVA::Types::kTesting);
loader->AddTree (backgroundTreeTrain,TString("Background"), weight2, cutstr, TMVA::Types::kTraining);
loader->AddTree (backgroundTreeTest,  TString("Background"), weight2, cutste, TMVA::Types::kTesting);

cout<<"ok  "<<endl;

// set weights for individual events
loader->SetSignalWeightExpression(weight_sig);
loader->SetBackgroundWeightExpression(weight_bkg);

//loader_prepare_flags = ["nTrain_Signal=0", "nTrain_Background=0", "NormMode=None", "!V"];
//options = ":".join(loader_prepare_flags);
//loader->PrepareTrainingAndTestTree(TCut(""), TCut(""), options) ;

//loader->PrepareTrainingAndTestTree( mycuts, mycutb, "nTrain_Signal=10210:nTrain_Background=95503:SplitMode=Alternate:NormMode=NumEvents:!V" );








DataSetInfo              : [dataset] : Added class "Signal"
                         : Add Tree test of type Signal with 8990 events
                         : Add Tree test of type Signal with 27314 events
DataSetInfo              : [dataset] : Added class "Background"
                         : Add Tree test of type Background with 467506 events
                         : Add Tree test of type Background with 1400556 events
ok  


# Booking Methods

Here we book the TMVA methods. We book a Likelihood based on KDE, a Fischer discriminant and a BDT

In [5]:
// Likelihood ("naive Bayes estimator")
//factory.BookMethod(loader, TMVA::Types::kLikelihood, "Likelihood",
//                           "H:!V:TransformOutput:PDFInterpol=Spline2:NSmoothSig[0]=20:NSmoothBkg[0]=20:NSmoothBkg[1]=10:NSmooth=1:NAvEvtPerBin=50" );

// Use a kernel density estimator to approximate the PDFs
//factory.BookMethod(loader, TMVA::Types::kLikelihood, "LikelihoodKDE",
//                           "!H:!V:!TransformOutput:PDFInterpol=KDE:KDEtype=Gauss:KDEiter=Adaptive:KDEFineFactor=0.3:KDEborder=None:NAvEvtPerBin=50" ); 


// Fisher discriminant (same as LD)
//factory.BookMethod(loader, TMVA::Types::kFisher, "Fisher", "H:!V:Fisher:VarTransform=None:CreateMVAPdfs:PDFInterpolMVAPdf=Spline2:NbinsMVAPdf=50:NsmoothMVAPdf=10" );


//Boosted Decision Trees
factory.BookMethod(loader,TMVA::Types::kBDT, "BDT",
                   "!V:NTrees=300:MinNodeSize=2.5%:MaxDepth=4:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );

//Multi-Layer Perceptron (Neural Network)
//factory.BookMethod(loader, TMVA::Types::kMLP, "MLP",
//                   "!H:!V:NeuronType=tanh:VarTransform=N:NCycles=100:HiddenLayers=N+5:TestRate=5:!UseRegulator" );

Factory                  : Booking method: BDT
                         : 
                         : Building event vectors for type 0 Signal
                         : Dataset[dataset] :  create input formulas for tree test
                         : Building event vectors for type 1 Signal
                         : Dataset[dataset] :  create input formulas for tree test
                         : Building event vectors for type 0 Background
                         : Dataset[dataset] :  create input formulas for tree test
                         : Building event vectors for type 1 Background
                         : Dataset[dataset] :  create input formulas for tree test
DataSetFactory           : [dataset] : Number of events in input trees
                         : 
                         : 
                         : Dataset[dataset] : Weight renormalisation mode: "EqualNumEvents": renormalises all event classes ...
                         : Dataset[dataset] :  such that t

### Booking Deep Neural Network

Here we book the new DNN of TMVA. If using master version you can use the new DL method

In [6]:
factory.OptimizeAllMethods("ROCIntegral","FitGA");

                         : Optimize method: BDT for Classification
                         :  the following BDT parameters will be tuned on the respective *grid*
                         : 
<WARNING>                : AdaBoostBeta
                         : | 0.2 || 0.4 || 0.6 || 0.8 || 1 |
<WARNING>                : MaxDepth
                         : | 2 || 3 || 4 |
<WARNING>                : MinNodeSize
                         : | 1 || 1.12444 || 1.26436 || 1.42169 || 1.5986 || 1.79753 || 2.02121 || 2.27272 || 2.55553 || 2.87354 || 3.23111 || 3.63318 || 4.08529 || 4.59365 || 5.16527 || 5.80802 || 6.53076 || 7.34343 || 8.25722 || 9.28473 || 10.4401 || 11.7392 || 13.2 || 14.8426 || 16.6896 || 18.7664 || 21.1016 || 23.7274 || 26.68 || 30 |
<WARNING>                : NTrees
                         : | 10 || 257.5 || 505 || 752.5 || 1000 |
                         : Automatic optimisation of tuning parameters in BDT uses:
                         : AdaBoostBeta in range from: 0.2 to: 1

In [7]:
bool useDL = false; 


#### 1. Define DNN layout

The DNN configuration os defined using a string. Note that whitespaces between characters are not allowed. 

We define first the DNN layout: 

- **input layout** :   this defines the input data format for the DNN as  ``input depth | height | width``. 
   In case of a dense layer as first layer the input layout should be  ``1 | 1 | number of input variables`` (features)
- **batch layout**  : this defines how are the input batch. It is related to input layout but not the same. 
   If the first layer is dense it should be ``1 | batch size ! number of variables`` (fetures)
   
   *(note the use of the character `|` as  separator of  input parameters for DNN layout)*
                 
- **layer layout** string defining the layer architecture. The syntax is  
   - layer type (e.g. DENSE, CONV, RNN)
   - layer parameters (e.g. number of units)
   - activation function (e.g  TANH, RELU,...)
   
     *the different layers are separated by the ``","`` *
                
#### 2. Define Trainining Strategy

We define here the training strategy parameters for the DNN. The parameters are separated by the ``","`` separator. 
One can then concatenate different training strategy with different parameters. The training strategy are separated by 
the ``"|"`` separator. 

 - Optimizer
 - Learning rate
 - Momentum (valid for SGD and RMSPROP)
 - Regularization and Weight Decay 
 - Dropout 
 - Max number of epochs 
 - Convergence steps. if the test error will not decrease after that value the training will stop
 - Batch size (This value must be the same specified in the input layout)
 - Test Repetitions (the interval when the test error will be computed) 


#### 3. Define general DNN options

We define the general DNN options concateneting in the final string the previously defined layout and training strategy.
Note we use the ``":"`` separator to separate the different higher level options, as in the other TMVA methods. 
In addition to input layout, batch layout and training strategy we add now: 

- Type of Loss function (e.g. cross entropy)
- Weight Initizalization (e.g XAVIER, XAVIERUNIFORM, NORMAL )
- Variable Transformation
- Type of Architecture (e.g. CPU, GPU, Standard)

We can then book the DL method using the built otion string
                

In [8]:
if (useDL) {  
     // Define DNN layout
     TString inputLayoutString = "InputLayout=1|1|25"; 
     TString batchLayoutString= "BatchLayout=1|128|25";
     TString layoutString ("Layout=DENSE|64|TANH,DENSE|64|TANH,DENSE|64|TANH,DENSE|64|TANH,DENSE|1|LINEAR");
                                                                                                                                                                                    
      // Define Training strategies 
      // one can catenate several training strategies 
      TString training1("LearningRate=1e-3,Momentum=0.9,Repetitions=1,"
                        "ConvergenceSteps=10,BatchSize=128,TestRepetitions=1,"
                        "MaxEpochs=30,WeightDecay=1e-4,Regularization=None,"
                        "Optimizer=ADAM,DropConfig=0.0+0.0+0.0+0.");
 //     TString training2("LearningRate=1e-3,Momentum=0.9,Repetitions=1,"
 //                       "ConvergenceSteps=10,BatchSize=128,TestRepetitions=1,"
 //                       "MaxEpochs=20,WeightDecay=1e-4,Regularization=None,"
 //                       "Optimizer=SGD,DropConfig=0.0+0.0+0.0+0.");
  
      TString trainingStrategyString ("TrainingStrategy=");
      trainingStrategyString += training1; // + "|" + training2;

      // General Options.                                                                                                                                                                
      TString dnnOptions ("!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=G:"
                          "WeightInitialization=XAVIER");
      dnnOptions.Append (":"); dnnOptions.Append (inputLayoutString);
      dnnOptions.Append (":"); dnnOptions.Append (batchLayoutString);
      dnnOptions.Append (":"); dnnOptions.Append (layoutString);
      dnnOptions.Append (":"); dnnOptions.Append (trainingStrategyString);

      dnnOptions += ":Architecture=CPU";
      factory.BookMethod(loader, TMVA::Types::kDL, "DL_CPU", dnnOptions);

}

## Train Methods

Here we train all the previously booked methods. 

In [9]:
factory.TrainAllMethods();

Factory                  : Train all methods
Factory                  : Train method: BDT for Classification
                         : 
<WARNING>                : Events with negative event weights are USED during the BDT training. This might cause problems with small node sizes or with the boosting. Please remove negative events from training using the option *IgnoreEventsWithNegWeightsInTraining* in case you observe problems with the boosting
BDT                      : #events: (reweighted) sig: 238248 bkg: 238248
                         : #events: (unweighted) sig: 8990 bkg: 467506
                         : Training 1000 Decision Trees ... patience please
                         : Elapsed time for training with 476496 events: 605 sec         
BDT                      : [dataset] : Evaluation of BDT on training sample (476496 events)
                         : Elapsed time for evaluation of 476496 events: 42.5 sec       
                         : Creating xml weight file: datase

## Test  all methods

Here we test all methods using the test data set

In [10]:
factory.TestAllMethods();   

Factory                  : Test all methods
Factory                  : Test method: BDT for Classification performance
                         : 
BDT                      : [dataset] : Evaluation of BDT on testing sample (1427870 events)
                         : Elapsed time for evaluation of 1427870 events: 108 sec       


In [ ]:
factory.EvaluateAllMethods();

## Plot ROC Curve
We enable JavaScript visualisation for the plots

In [ ]:
c1 = factory.GetROCCurve(loader);
c1->Draw();


####  Close outputfile to save all output information (evaluation result of methods)

In [ ]:
outputFile->Close();